Extracción de datos con Selenium

In [1]:
import pandas as pd  
import numpy as np
import re
from unidecode import unidecode # para quitar acentos --> 'pip install unicode' en la terminal si no está instalada

from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

import random

from time import sleep  

pd.set_option('display.max_columns', None)  

In [2]:
random.randint(0 , 100)

77

In [5]:
pelis_drama_2000_2004 = pd.read_csv("pelis_drama_2000_2004.csv")

In [6]:
ultimo_elemento = pelis_drama_2000_2004.shape[0]

pelis_buscar = [(pelis_drama_2000_2004.at[i, '1'] , pelis_drama_2000_2004.at[i, '2']) for i in range (0, ultimo_elemento)] # Se agrega a la lista una tupla con el nombre , año de la peli por cada índice dentro del rango 0 , ultimo elemento de pelis_drama_2000_2004

In [56]:
driver = webdriver.Chrome()

driver.get("https://www.rottentomatoes.com/")

# Ampliar la ventana a tamaño completo
driver.maximize_window()

sleep(3)

# Aceptar los anuncios en la página para continuar navegando
try:
    driver.find_element("xpath" , "/html/body/div[5]/div/div[1]/div/div/div/div[2]/div/div/button").click()
    
except:
    pass

# Rechazar cookies
try:
    driver.find_element("css selector" , "#onetrust-reject-all-handler").click()
    
except:
    pass

sleep(3)

ultimo_rango = len(pelis_buscar)

pelis_con_puntuacion = []

for i in range (0 , 20): # Limitamos el rango para poder probar si funciona
    
    nombre_peli = f'{pelis_buscar[i][0]}'
    anio_peli = int(f'{pelis_buscar[i][1]}')
    
    # Accedemos a la barra de búsqueda y buscamos el nombre de la peli
    driver.find_element("css selector", "#header-main > search-algolia > search-algolia-controls > input").send_keys(nombre_peli, Keys.ENTER)

    for resultado in range (1 , 30): # Itera por los primeros 30 resultados de la primera página
        
        try:
            datos_resultado = driver.find_element('css selector', f'#search-results > search-page-result:nth-child(2) > ul > search-page-media-row:nth-child({resultado})').text.split("\n") # Obtiene los datos del resultado (puntuacion, titulo, año...)
            titulo = re.search('(.+?)\s*\(\d{4}\)' , datos_resultado[1]).group(1) # Sacamos sólo el título de la combinación titulo-año
            anio = int(re.search('\((\d{4})\)' , datos_resultado[1]).group(1)) # Sacamos sólo el año de la combinación titulo-año
            
        except:
            break # Si no existe el resultado, detiene el bucle y busca otra película
        
        # Si el título se encuentra en el nombre de la película y el año es el mismo    
        if unidecode(titulo).lower() in unidecode(nombre_peli).lower() and anio_peli == anio: 
            
            # Prueba a añadir la puntuación como integer y en caso de que no exista, añade '--'
            try:
                puntuacion = int(datos_resultado[0].replace("%" , ""))
            except:
                puntuacion = '-'
                
            break  # Detiene el bucle al haber encontrado una coincidencia  
        
        # Si el título no coincide con el resultado       
        else:
            puntuacion = np.nan
        
    sleep(2)        
    tupla = (nombre_peli , anio_peli , puntuacion)
    pelis_con_puntuacion.append(tupla) 
         

In [57]:
pelis_con_puntuacion # Si aparecen con nan es que no las encontró - Si tienen '-' es que las encontró, pero no tenían puntuación - Si tienen un número, es que las encontró y tenían puntuación

[('Istota', 2000, nan),
 ('For the Cause', 2000, nan),
 ('Florentino y el diablo', 2000, nan),
 ('Karobaar: The Business of Love', 2000, nan),
 ('Gang', 2000, '-'),
 ('Consequence', 2000, nan),
 ('Fa yeung nin wah', 2000, nan),
 ('Beyond the Pale', 2000, nan),
 ('Chinese Coffee', 2000, '-'),
 ('Passionnément', 2000, '-'),
 ('Love 101', 2000, nan),
 ('Pushing the Envelope', 2000, nan),
 ('Way Past Cool', 2000, nan),
 ('State and Main', 2000, nan),
 ('Vulgar', 2000, 23),
 ('Bruno', 2000, 20),
 ('Lured Innocence', 2000, nan),
 ('Shikari', 2000, '-'),
 ('Capitães de Abril', 2000, nan),
 ('Sånger från andra våningen', 2000, nan)]